In [4]:
# read data
# 数据读取
import csv

# 读取filename里面的数据
def readFromFile(filename):
    dataLines = []
    with open(filename, newline='',encoding='utf-8') as f:
#         remove the first line
#         去除第一行
        f.readline()
        rows = csv.reader(f)
        dataLines = list(rows)
#         correct the label into standard form
#         将字符型label转换为整数型label，并且补全将‘’补成0
        for dataLine in dataLines:
            if dataLine[0] == '0.0' or dataLine[0] == '':
                dataLine[0] = 0
            elif dataLine[0] == '1.0':
                dataLine[0] = 1
            
            if dataLine[1] == '0.0' or dataLine[1] == '':
                dataLine[1] = 0
            elif dataLine[1] == '1.0':
                dataLine[1] = 1
    return dataLines

filenames = ['data/part1.csv', 'data/part2.csv', 'data/part4.csv']
dataLines = []
for filename in filenames:
    dataLines_ = readFromFile(filename)
    dataLines += dataLines_

In [2]:
# 查看最后一行数据
print(dataLines[599])

[0, 1, '多次购买，经济实惠，尺寸合适']


In [3]:
# comment data preprocessing
# 评论数据的预处理
## 0. shuffle data
## 0. 打乱数据顺序（需要注意由于数据打乱了，每次执行的结果都会不同）
from random import shuffle
shuffle(dataLines)

In [4]:
## 1. remove default comment
## 1. 去除默认评论
dataLines1 = [line for line in dataLines if line[2] != '此用户没有填写评论!']
print(len(dataLines1))      # 查看剩余数据量

572


In [5]:
## 2. remove noise
## 2. 移除噪声符号（可以修改，见文件word2vec/utils.py）
## [参考](https://blog.csdn.net/mach_learn/article/details/41744487)
import re
from word2vec.utils import remove_noise

dataLines2 = [[line[0], line[1], remove_noise(line[2])] for line in dataLines1]
print(dataLines2[0])        # 随机查看一条数据

[0, 0, '给了好评对不起自己收到的东西不给好评对不起下图销售人员保持沉默不作评论']


In [6]:
## 3. text segmentation
## 3. 使用jieba分词
import jieba

def segmentation(text):
    return jieba.cut(text)

dataLines3 = [[line[0], line[1], segmentation(line[2])] for line in dataLines2]

In [7]:
## 4. remove stop words
## 4. 移除停用词

# 加载停用词
def loadStopWords(filename):
    stopwords = [line.strip() for line in \
                 open(filename, 'r', encoding='utf-8').readlines()]
    return stopwords

stopWordFilename = 'stop_words'
stopWords = loadStopWords(stopWordFilename)
dataLines4 = [[line[0], line[1], [word for word in line[2] if word not in stopWords]]
             for line in dataLines3]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.598 seconds.
Prefix dict has been built succesfully.


In [8]:
## 5. word Embedding, the word2Vec training see folder word2vec
## 5. 词向量嵌入，这里加载训练好的word2vec模型，训练过程见文件夹word2vec
## Currently the corpus is not cleaned yet.
## 现在训练word2vec模型的语料预处理同前面一样，调用remove_noise()和结巴分词
import gensim

# 加载训练好的模型
model = gensim.models.Word2Vec.load('word2vec/w2v.model')

# 将文本转换为词向量
dataLines5 = []
for line in dataLines4:
    temp = []
    temp = [model.wv[word] for word in line[2] if word in list(model.wv.vocab)]
    if len(temp) == 0:
        temp = [0 for i in range(100)]
    else:
        temp = sum(temp)
    dataLines5.append([line[0], line[1], temp])
dataLines5[2]

[0, 0, array([-0.5696369 ,  1.434681  , -1.3457062 , -1.0737123 ,  1.7944949 ,
        -1.8746401 , -1.7441665 , -0.33646914, -1.3398262 , -0.613251  ,
         3.7658167 , -0.81488925, -2.2925906 ,  3.2544887 ,  0.47307578,
         2.6945121 , -0.25653872,  1.683943  ,  2.5718057 ,  3.3660645 ,
        -2.7731285 ,  0.76614094,  0.29850632,  1.1424104 , -1.0080081 ,
         1.1021123 , -0.8836053 ,  2.5620518 , -1.0686977 ,  2.7273965 ,
        -2.1284513 , -1.0426029 , -0.11974853, -5.3148174 ,  3.2486863 ,
        -1.5224168 ,  0.2926951 ,  0.46637464,  1.9785173 ,  0.98617196,
        -3.0494134 , -0.34304577, -0.7633401 ,  1.0476135 , -0.3061484 ,
         2.9846144 ,  0.78348136,  3.5827913 , -0.6293336 ,  0.9463711 ,
        -0.17951652, -0.41948265,  1.7393503 , -0.47981304, -1.4905112 ,
         1.8823681 , -0.39938334,  0.5702089 , -0.76978004, -1.8762641 ,
         1.9003918 , -0.8907394 , -2.0959423 , -2.5957978 , -0.11157587,
         0.23382859,  0.5976219 ,  2.2379594 

In [9]:
# logistic regression
# logistic回归

from sklearn.linear_model import LogisticRegression
import numpy as np

# 将dataLines5的数据，label分开
X_all = [line[2] for line in dataLines5]
Y1_all = [line[0] for line in dataLines5]
Y2_all = [line[1] for line in dataLines5]

# 划分数据，前500条为训练集，后72条为测试集
X_train = X_all[0:500]
Y1_train = Y1_all[0:500]
Y2_train = Y2_all[0:500]

X_test = X_all[500:]
Y1_test = Y1_all[500:]
Y2_test = Y2_all[500:]

In [11]:
clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train, Y1_train)
clf.score(X_test, Y1_test)

0.9861111111111112

In [22]:
# print data predict as template comment
# 打印被分类为模板的评论
Y1_predict = clf.predict(X_test)
for i in range(len(X_test)):
    if Y1_predict[i] == 1:
        print(dataLines1[i+500][2])
        print()

【因为本仙女很懒，不想每个产品都写好评，所以特地模仿好友写下这个模板，但是这个产品无论是质量还是外形都是本仙女喜欢的类型，如果不喜欢，仙女收到东西会很生气，然后这个模板就会成为仙女喋喋不休的休书，自然不可能撒下这个好评，给各位淘友一个参考，本产品还是极好的。 &mdash;&mdash;来自一位慵懒的只爱购物不爱写评语却想换积分的仙女！ 好吧，说真的，很好，喜欢。】

宝贝已经收到了，真的是物有所值非常的满意。卖家的服务态度很好发货速度也很快，包裹的严严实实没有任何破损。快递小哥送货速度快，总体来说是一次愉快的购物呀，下次有需要还会再来买买买！与卖家描述的完全一致，非常满意,真的很喜欢，完全超出期望值，发货速度非常快今天收到快递，首先我第一次体验到神速的快递，昨天晚上买的，今天早上到了，其次看见我的商品，我觉得物美价廉非常的nice，天猫超市永远是正品，值得信赖，所以我的配图是爱心，因为我觉得很好很棒，很值得去再回购

宝宝们！！！半,价入手滴！公～总～号 ：【真满意】再买，便宜了一半！！超级好！！！东西很好，价美物廉，质量杠杠滴，试穿了一下，舒适，于是就又买了一件，大小合适，样式也蛮喜欢的。身高170体重75公斤这个码很合适，颜色好看。衣服上身很好看，尺寸刚好，觉得好看，才买的，客服推荐的尺码穿上也特别的合适，我比较喜欢简单大方的衣服，上身效果也不错回复也很快，我问了不少问题，物流也比较快，衣服的面料比较柔软穿上很舒服，衣服面料好，物流也挺快的，也要用心敷衍吧，取到快递，客服服务态度好，衣服料子很好，

宝宝们！！半，价买滴哟！公?总，号搜：【太实惠】再去买，超级划算！！！！排在最上面的那个才是哦！！收到衣服，就迫不及待的打开看了。第一次在米兔家买衣服，真心好呀，质量款式都好，也很厚实！试穿也好合身哦！很适合我的风格！ 这个颜色真是如描述超柔和的！而且很提肤色哦！这是目前买到的最满意的一件哟！喜欢！喜欢！快点冷起来吧！ 宝贝非常漂亮 和描述的一样 做工精致 款式漂亮 卖家服务非常好 耐心细致 包装非常讲究 不愧是品牌呀 还会在来的 物流也很给力 谢谢了 好评



In [23]:
# print the conflict data in test set
# 打印模板评论分类错误的测试数据，便于分析

def print_error_data(X_t, Y_t, Y_predict):
    for i in range(len(X_t)):
        if (Y_predict[i] != Y_t[i]):
            print(dataLines1[i+500][2])
            print("True label %d" % Y_t[i])
            print("Predict label %d" % Y_predict[i])
            print()
            
print_error_data(X_test, Y1_test, Y1_predict)
# 可以看到，这里虽然标注为0，但是如果是我，我会标注为1（再次执行可能结果不同）

宝贝已经收到了，真的是物有所值非常的满意。卖家的服务态度很好发货速度也很快，包裹的严严实实没有任何破损。快递小哥送货速度快，总体来说是一次愉快的购物呀，下次有需要还会再来买买买！与卖家描述的完全一致，非常满意,真的很喜欢，完全超出期望值，发货速度非常快今天收到快递，首先我第一次体验到神速的快递，昨天晚上买的，今天早上到了，其次看见我的商品，我觉得物美价廉非常的nice，天猫超市永远是正品，值得信赖，所以我的配图是爱心，因为我觉得很好很棒，很值得去再回购
True label 0
Predict label 1



In [13]:
clf2 = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train, Y2_train)
clf2.score(X_test, Y2_test)

0.9027777777777778

In [16]:
# print the conflict data in test set
# 打印水评分类错误的测试数据，便于分析

print_error_data(X_test, Y2_test, Y2_predict)

【因为本仙女很懒，不想每个产品都写好评，所以特地模仿好友写下这个模板，但是这个产品无论是质量还是外形都是本仙女喜欢的类型，如果不喜欢，仙女收到东西会很生气，然后这个模板就会成为仙女喋喋不休的休书，自然不可能撒下这个好评，给各位淘友一个参考，本产品还是极好的。 &mdash;&mdash;来自一位慵懒的只爱购物不爱写评语却想换积分的仙女！ 好吧，说真的，很好，喜欢。】
True label 0
Predict label 1

买回来的颜色跟展示图片的都不一样
True label 0
Predict label 1

衣服好
True label 1
Predict label 0

呱呱呱呱呱呱嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎嘎非常好，里面还有加绒。  哈哈哈哈哈嘎嘎嘎嘎
True label 1
Predict label 0

宝宝们！！半，价买滴哟！公?总，号搜：【太实惠】再去买，超级划算！！！！排在最上面的那个才是哦！！收到衣服，就迫不及待的打开看了。第一次在米兔家买衣服，真心好呀，质量款式都好，也很厚实！试穿也好合身哦！很适合我的风格！ 这个颜色真是如描述超柔和的！而且很提肤色哦！这是目前买到的最满意的一件哟！喜欢！喜欢！快点冷起来吧！ 宝贝非常漂亮 和描述的一样 做工精致 款式漂亮 卖家服务非常好 耐心细致 包装非常讲究 不愧是品牌呀 还会在来的 物流也很给力 谢谢了 好评
True label 0
Predict label 1

挺好看的舒服实费和团队采访了107个家庭、翻遍近几十年美国社会的各种统计资料后，帕特南得出了一个残酷的结论：在美国，阶层流动几近停顿，穷人再努力，也是出头无望。  情形有多严重呢？我们可以从一个侧面感受下
True label 1
Predict label 0

跟图片一样，还不错，这个价位买到一件不错的衣服
True label 0
Predict label 1



In [20]:
# check how many comment in test set appeared in training set
seen_count = 0
for data_test in X_test:
    for data_train in X_train:
        if set(data_train) == set(data_test):
            seen_count += 1
            break
print(seen_count)    
# 可以看到出现在训练集的测试集数据只有1条（每次执行结果不同，重新执行会改变）

1


In [15]:
# 可以调用其他模型训练 or 可以使用评估方法对实验结果进行评估
# ...